In [1]:
import datetime
import re
import time

import torch
import numpy as np
import pandas as pd
import pickle
import json
from tqdm import tqdm
from scipy import sparse
from sklearn.linear_model import LogisticRegression
import scipy.stats

In [2]:
with open('results.pkl', 'rb') as results_file:
    raw_results = pickle.load(results_file)

In [3]:
with open('tournaments.pkl', 'rb') as tournaments_file:
    raw_tournaments = pickle.load(tournaments_file)

In [4]:
with open('players.pkl', 'rb') as players_file:
    raw_players = pickle.load(players_file)

# Задание 1

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). 

Для унификации предлагаю:

* взять в тренировочный набор турниры с dateStart из 2019 года; 

* в тестовый — турниры с dateStart из 2020 года.


Начнем с того, что из raw_results выкинем все турниры раньше 2019 года, содержащие данные о составах комманд и повопросные результаты.

In [5]:
tournament_keys = [v['id'] for k, v in raw_tournaments.items() if v['dateStart'][:4] == '2019' or v['dateStart'][:4] == '2020']

for key in tqdm(list(raw_results.keys())):
    if key not in tournament_keys or raw_results[key] == [] or not raw_results[key][0].get('mask', None) or raw_results[key][0]['mask'] == None:
        raw_results.pop(key, None)
        
len(raw_results)

100%|██████████| 5528/5528 [00:01<00:00, 3565.70it/s]


847

Удалим команды с масками ответов не равными максимальной в соревновании, удалим вопросы с пометкой `X` и заменим вопросы с пометкой `?` на `0`.

In [6]:
def check_if_mask_empty(results):
    #Проверка на наличие маски в результатах
    for tournament, teams in results.items():
        for team in teams:
            if not team['mask']:
                return True
    return False

In [7]:
def check_if_mask_small(results):
    #проверка длины маски в соревновании
    for tournament, teams in results.items():
        max_mask_lengt = 0
        for team in teams:
            if team['mask'] and len(team['mask']) > max_mask_lengt:
                max_mask_lengt = len(team['mask'])
        for team in teams:
            if team['mask'] and len(team['mask']) < max_mask_lengt:
                return True
    return False

In [8]:
def check_X_in_masks(results):
    #Проверка на наличие 'X' в результатах
    for tournament, teams in results.items():
        for team in teams:
            if team['mask'] and 'X' in team['mask']:
                return True
    return False

In [9]:
def check_qmark_in_masks(results):
    #Проверка на наличие '?' в результатах
    for tournament, teams in results.items():
        for team in teams:
            if team['mask'] and '?' in team['mask']:
                return True
    return False

In [10]:
def clean_masks(results):
    for tournament, teams in tqdm(results.items()):
        max_mask_lengt = 0
        #удалим из масок 'X' и заменим '?' на '0', а так же найдем максимальную маску в соревновании
        for team in teams:
            if team['mask'] and 'X' in team['mask']:
                team['mask'] = team['mask'].replace('X', '')
            if team['mask'] and '?' in team['mask']:
                team['mask'] = team['mask'].replace('?', '0')
            if team['mask'] and len(team['mask']) > max_mask_lengt:
                max_mask_lengt = len(team['mask'])
        #удалим команды с пустыми полями 'mask' и с короткими масками
        for idx, team in list(enumerate(teams))[::-1]:
            if not team['mask'] or len(team['mask']) < max_mask_lengt:
                teams.pop(idx)

Запустим чекеры до очистки данных:

In [11]:
print(check_if_mask_empty(raw_results))
print(check_if_mask_small(raw_results))
print(check_X_in_masks(raw_results))
print(check_qmark_in_masks(raw_results))

True
True
True
True


Запустим процесс очистки:

In [12]:
clean_masks(raw_results)

100%|██████████| 847/847 [00:00<00:00, 2602.34it/s]


Теперь чекеры не находят проблем:

In [13]:
print(check_if_mask_empty(raw_results))
print(check_if_mask_small(raw_results))
print(check_X_in_masks(raw_results))
print(check_qmark_in_masks(raw_results))

False
False
False
False


Составим обучающую и тестовую выборки по турнирам:

In [14]:
tournaments_train = []
tournaments_test = []

for tournament_id in tqdm(raw_results.keys()):
    tournament = dict()
    tournament['id'] = tournament_id
    teams_list = []
    for team in raw_results[tournament_id]:
        team_info = dict()
        team_info['id'] = team['team']['id']
        team_info['mask'] = team['mask']
        team_info['players'] = [member['player']['id'] for member in team['teamMembers']]
        teams_list.append(team_info)
    tournament['teams'] = teams_list
    
    if raw_tournaments[tournament_id]['dateStart'][:4] == '2019':
        tournaments_train.append(tournament)
    else:
        tournaments_test.append(tournament)

100%|██████████| 847/847 [00:00<00:00, 1035.39it/s]


In [15]:
with open('tournaments_train.json', 'w') as fout:
    json.dump(tournaments_train, fout)
    
with open('tournaments_test.json', 'w') as fout:
    json.dump(tournaments_test, fout)

In [16]:
with open('tournaments_train.json') as fin:
    tournaments_train = json.load(fin)
    
with open('tournaments_test.json') as fin:
    tournaments_test = json.load(fin)

In [17]:
len(tournaments_train), len(tournaments_test)

(674, 173)

Удалим исходные данные:

In [18]:
del raw_results, raw_tournaments

# Задание 2

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Присвоим каждому `id` игрока порядковый номер:

In [19]:
players = set()

for tournament in tqdm(tournaments_train):
    for team in tournament['teams']:
        for player in team['players']:
            players.add(player)

100%|██████████| 674/674 [00:00<00:00, 6906.58it/s]


In [20]:
id_to_player = {i: p for i, p in enumerate(players)}
player_to_id = {p: i for i, p in id_to_player.items()}

Составим таблицу повопросных ответов игроков размерности (все игроки + все вопросы) * (повопросные ответы).

Целевая переменная - правильность ответа (`0` или `1`)

In [21]:
%%time
player_ids = []
question_ids = []
y_labels = []

team_ids = []

questions_start_index = len(player_to_id)
for tournament in tqdm(tournaments_train):
    for team in tournament['teams']:
        team_mask = team['mask']
        for q, answer in enumerate(team_mask, questions_start_index):
            for player in team['players']:
                player_ids.append(player_to_id[player])
                question_ids.append(q)
                y_labels.append(int(answer))
                
                team_ids.append(team['id'])
                
    questions_start_index += len(team_mask)

100%|██████████| 674/674 [00:14<00:00, 48.08it/s] 

CPU times: user 13.6 s, sys: 465 ms, total: 14.1 s
Wall time: 14 s


In [22]:
y_train = np.array(y_labels)

In [23]:
%%time
X_train = sparse.lil_matrix((len(player_ids), questions_start_index), dtype=int)
X_train[range(len(player_ids)), player_ids] = 1
X_train[range(len(player_ids)), question_ids] = 1

len(player_ids), questions_start_index

CPU times: user 49 s, sys: 8.92 s, total: 57.9 s
Wall time: 57.8 s


(17751584, 90656)

In [24]:
sparse.save_npz('X_train_matrix.npz', X_train.tocoo())
np.savetxt('y_train_vec.txt', y_train)

In [25]:
X_train = sparse.load_npz('X_train_matrix.npz')
y_train = np.loadtxt('y_train_vec.txt')

In [26]:
X_train.shape, y_train.shape

((17751584, 90656), (17751584,))

Обучим на полученных данных логистическую регрессию:

In [27]:
base_model = LogisticRegression()

In [28]:
%%time
base_model.fit(X_train, y_train)

CPU times: user 4min 39s, sys: 2min 42s, total: 7min 21s
Wall time: 4min


/home/dkulemin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [29]:
with open('base_model', 'wb') as fout:
    pickle.dump(base_model, fout)

In [30]:
with open('base_model', 'rb') as fin:
    base_model = pickle.load(fin)

В качестве рейтинга игроков возьмем первые коэффициенты модели:

In [31]:
player_rating = base_model.coef_[0][:len(player_to_id)]

Составим рейтинг лист игроков:

In [32]:
rating_table = []

for i, rating in enumerate(player_rating):
    player_id = id_to_player[i]
    rating_table.append([rating, player_id, raw_players[player_id]['name'] + ' ' + raw_players[player_id]['surname']])

In [33]:
rating_list = pd.DataFrame(sorted(rating_table, reverse=True), columns=['rating', 'player id', 'player name'])
rating_list.head(30)

,rating,player id,player name
0,3.791533,27403,Максим Руссо
1,3.635847,4270,Александра Брутер
2,3.397761,30152,Артём Сорожкин
3,3.387160,28751,Иван Семушин
4,3.320266,27822,Михаил Савченков
5,3.261366,30270,Сергей Спешков
6,3.197049,20691,Станислав Мереминский
7,3.184139,34328,Михаил Царёв
8,3.158918,37047,Мария Юнгер
9,3.158539,18036,Михаил Левандовский


# Задание 3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


Оценим позицию команды в рейтинге, исходя из вероятности того, что хотябы один игрок команды правильно ответил на вопрос.

Для начала посчитаем число правильных ответов в командах:

In [34]:
tournaments_answers_count = []

for tournament in tqdm(tournaments_test):
    teams_answers_count = []
    for team in tournament['teams']:  
        test_player_ids = [player_to_id[player] for player in team['players'] if player in player_to_id.keys()]
        if len(test_player_ids):
            team_answers = list(map(int, team['mask']))
            teams_answers_count.append(sum(team_answers))
    tournaments_answers_count.append(teams_answers_count)

100%|██████████| 173/173 [00:00<00:00, 785.35it/s]


Теперь предскажем рейтинги команд, интересующие вероятности расчитаем по формуле:

$$P(team=1) = 1 - \prod P(player=0)$$

In [35]:
tournaments_rating_pred = []
for torunament in tqdm(tournaments_test):
    preds = []
    for team in torunament['teams']:
        test_player_ids = [player_to_id[player] for player in team['players'] if player in player_to_id.keys()]
        if len(test_player_ids):
            X = sparse.lil_matrix((len(test_player_ids), questions_start_index), dtype=int)
            X[range(len(test_player_ids)), test_player_ids] = 1

            fail_probas = base_model.predict_proba(X)[:, 0]
            team_proba = 1 - fail_probas.prod()
            preds.append(team_proba)
    tournaments_rating_pred.append(preds)

100%|██████████| 173/173 [00:12<00:00, 13.56it/s]


Расчитаем коеффициенты корреляции:

In [36]:
spearmanr_corrs = []
kendall_corrs = []
for i in range(len(tournaments_answers_count)):
    if len(tournaments_answers_count[i]) > 1:
        spearman = scipy.stats.spearmanr(tournaments_answers_count[i], tournaments_rating_pred[i]).correlation
        kendall = scipy.stats.kendalltau(tournaments_answers_count[i], tournaments_rating_pred[i]).correlation
        spearmanr_corrs.append(spearman)
        kendall_corrs.append(kendall)
print(f'Корреляция Спирмена: {np.mean(spearmanr_corrs):.4f}')
print(f'Корреляция Кендалла: {np.mean(kendall_corrs):.4f}')

Корреляция Спирмена: 0.7631
Корреляция Кендалла: 0.6084


# Задание 4

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.
